# Лабораторная работа №1


_ПИН-212 Жан-Лин Никита_

### Загрузка данных

In [1]:
# Импортируем необходимые библиотеки
# Импортируем классификатор K-ближайших соседей (KNN)
from sklearn.neighbors import KNeighborsClassifier
# Импортируем функции для для поиска гиперпараметров модели
from sklearn.model_selection import GridSearchCV
# Импортируем функции для оценки качества модели
from sklearn.metrics import classification_report
# Импортируем функции для разделения данных на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split
# Импортируем классы для стандартизации признаков
from sklearn.preprocessing import StandardScaler
# Pandas используется для работы с данными в табличном формате
import pandas as pd

In [4]:
# Читаем данные из CSV-файла
data = pd.read_csv('smoking_driking_dataset_Ver02.csv')
# Выводим первые 5 строк датасета
print(data.head())

# Выводим информацию о типах данных
print(data.info())


    sex  age  height  weight  waistline  sight_left  sight_right  hear_left  \
0  Male   35     170      75       90.0         1.0          1.0        1.0   
1  Male   30     180      80       89.0         0.9          1.2        1.0   
2  Male   40     165      75       91.0         1.2          1.5        1.0   
3  Male   50     175      80       91.0         1.5          1.2        1.0   
4  Male   50     165      60       80.0         1.0          1.2        1.0   

   hear_right    SBP  ...  LDL_chole  triglyceride  hemoglobin  urine_protein  \
0         1.0  120.0  ...      126.0          92.0        17.1            1.0   
1         1.0  130.0  ...      148.0         121.0        15.8            1.0   
2         1.0  120.0  ...       74.0         104.0        15.8            1.0   
3         1.0  145.0  ...      104.0         106.0        17.6            1.0   
4         1.0  138.0  ...      117.0         104.0        13.8            1.0   

   serum_creatinine  SGOT_AST  SGOT_AL

In [6]:
# Преобразуем 'sex' в числовую категорию
data['sex'] = data['sex'].map({'Male': 1, 'Female': 0}).astype(int)

# Преобразуем 'DRK_YN' в бинарные метки
data['DRK_YN'] = data['DRK_YN'].map({'Y': 1, 'N': 0}).astype(int)

# Проверка уникальных значений в целевой переменной
print("Уникальные значения в data['DRK_YN']:", data['DRK_YN'].unique())


Уникальные значения в data['DRK_YN']: [1 0]


In [8]:
missing_values = data.isnull().sum()
print("Пропущенные значения:\n", missing_values[missing_values > 0])

Пропущенные значения:
 Series([], dtype: int64)


In [10]:
# Создаем объект StandardScaler для масштабирования числовых признаков
scaler = StandardScaler()
# Выбираем все столбцы, которые имеют числовые типы данных (int32, int64, float32, float64)
numeric_features = data.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns
# Не включаем целевую переменную в нормализацию
features_to_scale = numeric_features.difference(['DRK_YN'])
# Применяем масштабирование ко всем числовым признакам и сохраняем их обратно в DataFrame
data[features_to_scale] = scaler.fit_transform(data[features_to_scale])

In [12]:
# Разделение данных на признаки и целевую переменную
X = data.drop('DRK_YN', axis=1) # Все столбцы, кроме 'DRK_YN', используются как признаки
Y = data['DRK_YN']              # 'DRK_YN' используется как целевая переменная

In [14]:
# Разделяем данные на обучающую и тестовую выборки
# test_size=0.2 означает, что 20% данных будет использовано для тестирования, а 80% для обучения
# random_state=42 используется для воспроизводимости результатов

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Проверка уникальных значений в Y_train и Y_test
print("Уникальные значения в Y_train:", Y_train.unique())
print("Уникальные значения в Y_test:", Y_test.unique())

Уникальные значения в Y_train: [0 1]
Уникальные значения в Y_test: [0 1]


### Провести классификацию найденного датасета, методом к- ближайших соседей.

In [16]:
# Определяем сетку параметров для поиска оптимальных гиперпараметров классификатора K-ближайших соседей (KNN)
# Определяем обширную сетку параметров, включая различные метрики расстояния и количество соседей
parameters = {
    'metric': ['hamming', 'minkowski', 'manhattan', 'braycurtis', 'correlation', 'nan_euclidean', 'cityblock', 'euclidean'],
    'n_neighbors': [5, 7, 9, 11, 13]
}

In [18]:
# Создаем экземпляр классификатора K-ближайших соседей (KNN)
knn = KNeighborsClassifier()

In [20]:
# Поиск оптимальных параметров с использованием кросс-валидации
grid_search = GridSearchCV(knn, parameters, cv=5)  # cv - количество фолдов для кросс-валидации
grid_search.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['hamming', 'minkowski', 'manhattan',
                                    'braycurtis', 'correlation',
                                    'nan_euclidean', 'cityblock', 'euclidean'],
                         'n_neighbors': [5, 7, 9, 11, 13]})

In [22]:
# Получение лучших параметров
best_params = grid_search.best_params_
print("Лучшие параметры:", best_params)

Лучшие параметры: {'metric': 'braycurtis', 'n_neighbors': 13}


In [24]:
# Обучение модели с лучшими параметрами
best_knn = grid_search.best_estimator_
best_knn.fit(X_train, Y_train)

KNeighborsClassifier(metric='braycurtis', n_neighbors=13)

In [26]:
# Предсказание на тестовом наборе данных
Y_pred = best_knn.predict(X_test)


In [28]:
# Оценка модели
report = classification_report(Y_test, Y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.72      0.69      0.71      3006
           1       0.70      0.73      0.72      2994

    accuracy                           0.71      6000
   macro avg       0.71      0.71      0.71      6000
weighted avg       0.71      0.71      0.71      6000

